<a href="https://colab.research.google.com/github/Ankit-Yadav22/Calamity_Alert_ML_pro/blob/main/Disaster_Alert_msg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"xxxx"})   # replace the xxxx in id with id of file you want to access
downloaded.GetContentFile('disaster.csv') 

In [ ]:
import pandas as pd
train_data = pd.read_csv('disaster.csv')

In [ ]:
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=12a2ef9207f87338e5a71196b104fba38680241bff6e6cce55385883dd636f2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ar3mqu3y/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
import numpy as np
import spacy
# Need to load the large model to get the vectors


texts = train_data.text.values
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in texts])
doc_vectors.shape

(7613, 300)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(doc_vectors,train_data.target ,
                                                    test_size=0.1, random_state=1)
from sklearn.svm import LinearSVC

# Set dual=False to speed up training, and it's not needed
model = LinearSVC(random_state=1, dual=False, max_iter=10000)
model.fit(X_train, y_train)
print(f"Accuracy: {model.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 80.840%


In [ ]:
import pickle

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)
# Load from file


In [ ]:
!ls

adc.json  disaster.csv	drive  pickle_model.pkl  sample_data


In [ ]:
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = pickle_model.score(X_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = pickle_model.predict(X_test)

Test score: 80.84 %


In [ ]:
file = drive.CreateFile({'parents':[{u'id':"12NpDvv-8UTww7E5q_iLlGhMSdBLFaWKL"}]})
file.SetContentFile('pickle_model.pkl')
file.Upload() 

In [ ]:
vec="the forest is on fire"
with nlp.disable_pipes():
    doc_vector = np.array([nlp(text).vector for text in vec])
print(doc_vector)
y=model.predict(doc_vector)
print(y)

[[-0.16832   0.48789  -0.30952  ...  0.11638   0.41141   0.18166 ]
 [-0.29438   1.2145   -0.69893  ...  0.081675 -0.12505   0.19485 ]
 [-0.25172  -0.26694  -0.50901  ... -0.25929   0.039081  0.61514 ]
 ...
 [ 0.18733   0.40595  -0.51174  ...  0.16495   0.18757   0.53874 ]
 [-0.23627   0.14954  -0.55652  ... -0.11957   0.12191   0.20341 ]
 [-0.25172  -0.26694  -0.50901  ... -0.25929   0.039081  0.61514 ]]
[0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0]


In [ ]:
from sklearn import svm
svmmodel = svm.SVC(kernel='linear', C =1)
svmmodel.fit(X_train, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
import pickle

# Save to file in the current working directory
pkl_filename1 = "pickle_modelsvm.pkl"
with open(pkl_filename1, 'wb') as file:
    pickle.dump(svmmodel, file)
# Load from file
with open(pkl_filename1, 'rb') as file:
    picklesvm = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = picklesvm.score(X_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))
Ypredict = picklesvm.predict(X_test)

Test score: 81.10 %


In [ ]:
text = "the forest is on fire"
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector])
Ypred =picklesvm.predict(doc_vectors)
print(Ypred)

[1]


In [ ]:
file = drive.CreateFile({'parents':[{u'id':"12NpDvv-8UTww7E5q_iLlGhMSdBLFaWKL"}]})
file.SetContentFile('pickle_modelsvm.pkl')
file.Upload() 